In [1]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np
import sys
import time
from sklearn.metrics import accuracy_score
from tqdm import tqdm_notebook as tqdm
import pickle

sys.path.insert(0, "../../")

from rcalgo.tf.models.semi_supervised.self_training import SelfTraining
from rcalgo.tf.models.semi_supervised.self_training import SelfTrainData
from rcalgo.tf.models.classifier.text_classifier import TextCNN
from rcalgo.tf.utils.tokenizers import CharTokenizer
from rcalgo.tf.training.hook import TensorInfoPrintHook
from rcalgo.tf.training.data.data_iterator import DataIterator

In [2]:
# load data
data_dir = "/home/web_server/antispam/project/datasets/female_vulgar/"
train_data = pd.read_csv(data_dir + 'train_1w.csv')
test_data = pd.read_csv(data_dir + 'test.csv')
# count word dict
tokenizer = CharTokenizer.build_from_corpus(test_data['text'], freq_threshold=2)
word_dict = tokenizer.word2idx

In [3]:
org_unsup_file = data_dir + "unsup0.txt"

In [5]:
sup_texts = train_data['text']
labels = train_data['label']
batch_size = 32
unsup_ratio = 1
gpu_num = 2
num_epochs = 150
num_train_steps = num_epochs * (int(len(sup_texts)) // (batch_size * gpu_num))
print(f"num_train_steps: {num_train_steps}")

num_train_steps: 23400


In [6]:
# create execute model
config = {
    # uda config
    "unsup_coeff": 1,  # unsup loss coeff 
    "batch_size": batch_size,
    "unsup_ratio": unsup_ratio,  # unsup data ratio
    "num_train_steps": num_train_steps - 200,  # num of train steps is needed for tas threshold calcuation
    
    # model config
    "learning_rate": 0.001,

    "num_classes": 2,
    "max_seq_len": 100,
    "embedding_size": 128,
    "num_layers": 2,
    "conv_num_filters": 256,
    "keep_prob": 0.4,
    
    "gpu_id": 0,
    "gpu_num": gpu_num,

    "model_output_dir": "./saved_models",
    "model_name": "cnn"
}

In [7]:
test_sup_texts = test_data['text']
test_labels = test_data['label']

In [8]:
org_unsup_texts = []
with open(org_unsup_file) as org_f:
    all_org_unsup_texts = [line.strip() for line in org_f.readlines()]

unsup_size = len(all_org_unsup_texts) // unsup_ratio
for i in range(unsup_size):
    org_unsup_texts.append([all_org_unsup_texts[i * unsup_ratio + j] for j in range(unsup_ratio)])
    
test_size = len(test_labels)
test_org_unsup_texts = org_unsup_texts[:test_size]

In [9]:
print(np.shape(org_unsup_texts))

(400000, 1)


In [10]:
print(f"sup_texts: ")
print(sup_texts[:5])
print(f"org_unsup_texts: ")
print(org_unsup_texts[:5])
print(len(org_unsup_texts))

sup_texts: 
0                上去不，宝贝
1            服务区这个小嘴太美了
2                 第二个雏吗
3       不要彩礼也不见得老公疼你一辈子
4    有没有发现大多都是女孩子患有抑郁症💔
Name: text, dtype: object
org_unsup_texts: 
[['祝雅儿新年快乐，开心快乐每一天，越来越漂亮哈[笑哭]'], ['非洲哥'], ['鬼街在哪啊'], ['哈卵'], ['差那点洗脸洗手的时间？']]
400000


## training with original model

In [10]:
tf.compat.v1.reset_default_graph()

teacher_model = TextCNN(config, word_dict, name="teacher_model", distribute="tf_distribute")
teacher_model.build_model()
print(teacher_model.model_summary())

2020-12-18 15:14:38,722  model.py (77) (MainThread) : INFO  config: {'unsup_coeff': 1, 'batch_size': 32, 'unsup_ratio': 2, 'num_train_steps': 23200, 'learning_rate': 0.001, 'num_classes': 2, 'max_seq_len': 100, 'embedding_size': 128, 'num_layers': 2, 'conv_num_filters': 256, 'keep_prob': 0.4, 'gpu_id': 0, 'gpu_num': 2, 'model_output_dir': './saved_models', 'model_name': 'cnn'}


Instructions for updating:
reduction_indices is deprecated, use axis instead

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

total parameter number: 581378
                         variable_name variable_shape parameters
0    embedding_layer/embedding_table:0    [3765, 128]     481920
1  my_conv_no.1/conv_1d/he_uniform/W:0  [1, 128, 128]      16384
2             my_conv_no.1/conv_1d/B:0          [128]        128
3  my_conv_no.2/conv_1d/he_uniform/W:0  [2, 128, 128]      32768
4             my_conv_no.2/conv_1d/B:0          [128]        128
5  my_conv_no.3/conv_1d/he_uniform/W:0  [3, 128, 128]      49152
6             my_conv_no.3/conv_1d/B:0          [128]        128
7                  fully_connected/W:0       [384, 2]        768
8                  fully_connected/B:0            [2]          2


In [11]:
teacher_model.train_and_evaluate(
    [sup_texts, labels], 
    [test_sup_texts, test_labels], 
    batch_size=batch_size * config['gpu_num'],
    epochs=30,
    hooks=[])

2020-12-18 15:14:39,135  single_worker_graph_copy_trainer.py (92) (MainThread) : INFO  Initializing trainer ...
2020-12-18 15:14:39,135  single_worker_graph_copy_trainer.py (105) (MainThread) : INFO  gpu_num: 2
2020-12-18 15:14:39,137  graph_copy.py (420) (MainThread) : INFO  build_train_graph kwargs: {'unsup_coeff': 1, 'batch_size': 32, 'unsup_ratio': 2, 'num_train_steps': 23200, 'learning_rate': 0.001, 'num_classes': 2, 'max_seq_len': 100, 'embedding_size': 128, 'num_layers': 2, 'conv_num_filters': 256, 'keep_prob': 0.4, 'model_output_dir': './saved_models', 'model_name': 'cnn', 'excluded_task_set': ['pretrained_model/default'], 'pretrained_model_scope': 'pretrained_model'}
2020-12-18 15:14:39,137  graph_copy.py (422) (MainThread) : INFO  build_train_graph: reconstructing variables...



Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


2020-12-18 15:14:40,587  graph_copy.py (444) (MainThread) : INFO  build_train_graph (teacher_model): building inputs...
2020-12-18 15:14:40,588  graph_copy.py (446) (MainThread) : INFO  task: teacher_model, <class 'str'>
2020-12-18 15:14:40,592  graph_copy.py (474) (MainThread) : INFO  build_train_graph (teacher_model): building per replica graph...



INFO:tensorflow:batch_all_reduce: 9 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10


2020-12-18 15:14:41,414  graph_copy.py (482) (MainThread) : INFO  train_ops: [PerReplica:{
  0 /replica:0/task:0/device:GPU:0: <tf.Tensor 'Adam_teacher_model/Identity:0' shape=() dtype=int64>,
  1 /replica:0/task:0/device:GPU:1: <tf.Tensor 'Adam_teacher_model/Identity_1:0' shape=() dtype=int64>
}], losses: [PerReplica:{
  0 /replica:0/task:0/device:GPU:0: <tf.Tensor 'Mean:0' shape=() dtype=float32>,
  1 /replica:0/task:0/device:GPU:1: <tf.Tensor 'replica_1/Mean:0' shape=() dtype=float32>
}], metrics: [PerReplica:{
  0 /replica:0/task:0/device:GPU:0: <tf.Tensor 'Cast:0' shape=(?,) dtype=float32>,
  1 /replica:0/task:0/device:GPU:1: <tf.Tensor 'replica_1/Cast:0' shape=(?,) dtype=float32>
}]


...

task:teacher_model, lr:1.0000e-03, loss:0.0149, metric:1.0000:  13%|█▎        | 21/156 [00:00<00:00, 208.23it/s] 

'epoch 27 train time(sec): 0.7515995502471924'
{'best_train_metrics': [0.9858],
 'best_validate_metrics': [0.8259],
 'epoch_index': 27,
 'train_loss': [0.0484],
 'train_metrics': [0.9827],
 'validate_loss': [0.8133],
 'validate_metrics': [0.8218]}


task:teacher_model, lr:1.0000e-03, loss:0.0315, metric:0.9844:  13%|█▎        | 21/156 [00:00<00:00, 204.91it/s] 

'epoch 28 train time(sec): 0.7425224781036377'
{'best_train_metrics': [0.9858],
 'best_validate_metrics': [0.8259],
 'epoch_index': 28,
 'train_loss': [0.0468],
 'train_metrics': [0.9847],
 'validate_loss': [0.8497],
 'validate_metrics': [0.8206]}


'epoch 29 train time(sec): 0.7486872673034668'
{'best_train_metrics': [0.9858],
 'best_validate_metrics': [0.8259],
 'epoch_index': 29,
 'train_loss': [0.0428],
 'train_metrics': [0.9858],
 'validate_loss': [0.8452],
 'validate_metrics': [0.8233]}


In [13]:
teacher_model.trainer.load_multi_replica_checkpoint("saved_models/best_model/teacher_model-epoch14.ckpt")

INFO:tensorflow:Restoring parameters from saved_models/best_model/teacher_model-epoch14.ckpt


In [14]:
unsup_res = teacher_model.predict([all_org_unsup_texts], batch_size=batch_size * config['gpu_num'])

  0%|          | 0/6250 [00:00<?, ?it/s]2020-12-18 15:16:56,650  single_worker_graph_copy_trainer.py (315) (MainThread) : INFO  task_input_tensors: {<training.trainer.Task object at 0x7f3a46a16cf8>: [<tf.Tensor 'input_text_plwd:0' shape=(?,) dtype=string>]}
2020-12-18 15:16:56,650  single_worker_graph_copy_trainer.py (316) (MainThread) : INFO  task_output_tensors: {<training.trainer.Task object at 0x7f3a46a16cf8>: [<tf.Tensor 'teacher_model/Softmax:0' shape=(?, 2) dtype=float32>]}


In [15]:
unsup_res_np = list(unsup_res.values())[0][0]

In [16]:
pseudo_label = np.argmax(unsup_res_np, axis=1)

In [17]:
pseudo_label[:2]

array([0, 0])

In [19]:
from collections import Counter
Counter(pseudo_label)

Counter({0: 187700, 1: 212300})

In [20]:
with open("pseudo_label.pkl", 'wb') as f:
    pickle.dump(pseudo_label, f)

## Self Training

In [11]:
import pickle

with open("pseudo_label.pkl", 'rb') as f:
    pseudo_label = pickle.load(f)

In [12]:
pseudo_label_final = []
unsup_size = len(pseudo_label) // unsup_ratio
for i in range(unsup_size):
    pseudo_label_final.append([pseudo_label[i * unsup_ratio + j] for j in range(unsup_ratio)])
    
print(np.shape(pseudo_label_final))

(400000, 1)


In [13]:
test_pseudo_label_final = pseudo_label_final[:test_size]

In [14]:
tf.compat.v1.reset_default_graph()
exec_model = TextCNN(config, word_dict=word_dict, name="cnn_model", distribute="tf_distribute")

2020-12-18 16:24:55,115  model.py (77) (MainThread) : INFO  config: {'unsup_coeff': 1, 'batch_size': 32, 'unsup_ratio': 1, 'num_train_steps': 23200, 'learning_rate': 0.001, 'num_classes': 2, 'max_seq_len': 100, 'embedding_size': 128, 'num_layers': 2, 'conv_num_filters': 256, 'keep_prob': 0.4, 'gpu_id': 0, 'gpu_num': 2, 'model_output_dir': './saved_models', 'model_name': 'cnn'}


In [15]:
student_self_training = SelfTraining(exec_model, config, name='student_self_trainig', distribute="tf_distribute")

2020-12-18 16:24:55,165  model.py (77) (MainThread) : INFO  config: {'unsup_coeff': 1, 'batch_size': 32, 'unsup_ratio': 1, 'num_train_steps': 23200, 'learning_rate': 0.001, 'num_classes': 2, 'max_seq_len': 100, 'embedding_size': 128, 'num_layers': 2, 'conv_num_filters': 256, 'keep_prob': 0.4, 'gpu_id': 0, 'gpu_num': 2, 'model_output_dir': './saved_models', 'model_name': 'cnn'}


In [16]:
student_self_training.build_model()
print(student_self_training.loss)
student_self_training.model_summary()


Instructions for updating:
reduction_indices is deprecated, use axis instead

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Tensor("semi-sup-loss:0", shape=(), dtype=float32)
total parameter number: 581378


variable_name variable_shape parameters
0    embedding_layer/embedding_table:0    [3765, 128]     481920
1  my_conv_no.1/conv_1d/he_uniform/W:0  [1, 128, 128]      16384
2             my_conv_no.1/conv_1d/B:0          [128]        128
3  my_conv_no.2/conv_1d/he_uniform/W:0  [2, 128, 128]      32768
4             my_conv_no.2/conv_1d/B:0          [128]        128
5  my_conv_no.3/conv_1d/he_uniform/W:0  [3, 128, 128]      49152
6             my_conv_no.3/conv_1d/B:0          [128]        128
7                  fully_connected/W:0       [384, 2]        768
8                  fully_connected/B:0            [2]          2

In [17]:
logging_hook = TensorInfoPrintHook({"unsup_loss": student_self_training.unsup_loss,
                                    "sup_loss": student_self_training.sup_loss})

In [18]:
student_self_training.trainer.initialize()

2020-12-18 16:25:03,886  single_worker_graph_copy_trainer.py (92) (MainThread) : INFO  Initializing trainer ...
2020-12-18 16:25:03,887  single_worker_graph_copy_trainer.py (105) (MainThread) : INFO  gpu_num: 2
2020-12-18 16:25:03,889  graph_copy.py (420) (MainThread) : INFO  build_train_graph kwargs: {'unsup_coeff': 1, 'batch_size': 32, 'unsup_ratio': 1, 'num_train_steps': 23200, 'learning_rate': 0.001, 'num_classes': 2, 'max_seq_len': 100, 'embedding_size': 128, 'num_layers': 2, 'conv_num_filters': 256, 'keep_prob': 0.4, 'model_output_dir': './saved_models', 'model_name': 'cnn', 'excluded_task_set': ['pretrained_model/default'], 'pretrained_model_scope': 'pretrained_model'}
2020-12-18 16:25:03,889  graph_copy.py (422) (MainThread) : INFO  build_train_graph: reconstructing variables...



Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


2020-12-18 16:25:05,365  graph_copy.py (444) (MainThread) : INFO  build_train_graph (student_self_trainig): building inputs...
2020-12-18 16:25:05,365  graph_copy.py (446) (MainThread) : INFO  task: student_self_trainig, <class 'str'>
2020-12-18 16:25:05,369  graph_copy.py (474) (MainThread) : INFO  build_train_graph (student_self_trainig): building per replica graph...



INFO:tensorflow:batch_all_reduce: 9 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10


2020-12-18 16:25:06,563  graph_copy.py (482) (MainThread) : INFO  train_ops: [PerReplica:{
  0 /replica:0/task:0/device:GPU:0: <tf.Tensor 'Adam_student_self_trainig/Identity:0' shape=() dtype=int64>,
  1 /replica:0/task:0/device:GPU:1: <tf.Tensor 'Adam_student_self_trainig/Identity_1:0' shape=() dtype=int64>
}], losses: [PerReplica:{
  0 /replica:0/task:0/device:GPU:0: <tf.Tensor 'semi-sup-loss:0' shape=() dtype=float32>,
  1 /replica:0/task:0/device:GPU:1: <tf.Tensor 'replica_1/semi-sup-loss:0' shape=() dtype=float32>
}], metrics: [PerReplica:{
  0 /replica:0/task:0/device:GPU:0: <tf.Tensor 'Cast:0' shape=(?,) dtype=float32>,
  1 /replica:0/task:0/device:GPU:1: <tf.Tensor 'replica_1/Cast:0' shape=(?,) dtype=float32>
}]


INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).


2020-12-18 16:25:06,610  single_worker_graph_copy_trainer.py (119) (MainThread) : INFO  all_tasks: ['student_self_trainig']
2020-12-18 16:25:06,641  single_worker_graph_copy_trainer.py (122) (MainThread) : INFO  init variables with sess target: None
2020-12-18 16:25:07,434  single_worker_graph_copy_trainer.py (124) (MainThread) : INFO  single_worker: initialized
2020-12-18 16:25:07,435  single_worker_graph_copy_trainer.py (125) (MainThread) : INFO  Init time cost (secs):  3.548


### pure self learning

In [17]:
train_data = SelfTrainData([sup_texts, org_unsup_texts, labels, pseudo_label_final],
                     mode="train", batch_size=batch_size * config['gpu_num'], 
                     drop_remainder=True, name="train_data")

test_data = SelfTrainData([test_sup_texts, test_org_unsup_texts, test_labels, test_pseudo_label_final],
                     mode="test", batch_size=batch_size * config['gpu_num'], 
                    drop_remainder=True, name="test_data")

2020-12-18 16:04:20,105  data.py (19) (MainThread) : INFO  train_data: will drop 16 remainder_size data items.
2020-12-18 16:04:20,463  data.py (19) (MainThread) : INFO  test_data: will drop 0 remainder_size data items.


In [18]:
np.shape(org_unsup_texts)

(66666, 6)

In [20]:
student_self_training.train_and_evaluate(train_data, test_data,
                batch_size=batch_size * config['gpu_num'],
                epochs=num_epochs,
                hooks=[logging_hook])

2020-12-18 16:04:24,756  single_worker_graph_copy_trainer.py (150) (MainThread) : INFO  tasks: ['student_self_trainig']
2020-12-18 16:04:24,757  single_worker_graph_copy_trainer.py (151) (MainThread) : INFO  loss: Tensor("Identity:0", shape=(), dtype=float32, device=/replica:0/task:0/device:CPU:0), loss_list: [<tf.Tensor 'Identity:0' shape=() dtype=float32>], task name: student_self_trainig
2020-12-18 16:04:24,757  single_worker_graph_copy_trainer.py (156) (MainThread) : INFO  do_test: True
2020-12-18 16:04:24,758  single_worker_graph_copy_trainer.py (157) (MainThread) : INFO  building dataset ...


sup_data_size: 9984, unsup_data_size: 66650


2020-12-18 16:04:25,009  single_worker_graph_copy_trainer.py (174) (MainThread) : INFO  start training sess run ...


last: {'unsup_loss': 0.37069973, 'sup_loss': 0.51862186}


task:student_self_trainig, lr:1.0000e-03, loss:0.8921, metric:0.7344:   8%|▊         | 12/156 [00:00<00:01, 112.03it/s]

...

last: {'unsup_loss': 0.15704007, 'sup_loss': 0.032739997}
'epoch 42 train time(sec): 1.2821848392486572'
{'best_train_metrics': [0.9921],
 'best_validate_metrics': [0.8302],
 'epoch_index': 42,
 'train_loss': [0.1632],
 'train_metrics': [0.9921],
 'validate_loss': [0.9432],
 'validate_metrics': [0.8269]}


task:student_self_trainig, lr:1.0000e-03, loss:0.1195, metric:1.0000:   0%|          | 0/156 [00:00<?, ?it/s]           

last: {'unsup_loss': 0.20486933, 'sup_loss': 0.18977433}
'epoch 43 train time(sec): 1.262631893157959'
{'best_train_metrics': [0.9921],
 'best_validate_metrics': [0.8302],
 'epoch_index': 43,
 'train_loss': [0.1649],
 'train_metrics': [0.9904],
 'validate_loss': [0.9436],
 'validate_metrics': [0.8288]}


last: {'unsup_loss': 0.16494687, 'sup_loss': 0.010225445}


KeyboardInterrupt: 

In my experiment, with 10000 training data of `female_vulgar`, the validate accuracy improved from **0.8259** to **0.8318**.

### Noisy Student

In [19]:
aug_unsup_file = data_dir + "unsup1.txt"

with open(aug_unsup_file) as aug_f:
    all_aug_unsup_texts = [line.strip() for line in aug_f.readlines()]

In [20]:
unsup_size = len(all_org_unsup_texts) // unsup_ratio
aug_unsup_texts = []

for i in range(unsup_size):
    aug_unsup_texts.append([all_aug_unsup_texts[i * unsup_ratio + j] for j in range(unsup_ratio)])
    
test_aug_unsup_texts = aug_unsup_texts[:test_size]

In [21]:
train_data = SelfTrainData([sup_texts, aug_unsup_texts, labels, pseudo_label_final],
                     mode="train", batch_size=batch_size * config['gpu_num'], 
                     drop_remainder=True, name="train_data")

test_data = SelfTrainData([test_sup_texts, test_aug_unsup_texts, test_labels, test_pseudo_label_final],
                     mode="test", batch_size=batch_size * config['gpu_num'], 
                    drop_remainder=True, name="test_data")

2020-12-18 16:25:09,613  data.py (19) (MainThread) : INFO  train_data: will drop 16 remainder_size data items.
2020-12-18 16:25:10,483  data.py (19) (MainThread) : INFO  test_data: will drop 0 remainder_size data items.


In [22]:
student_self_training.train_and_evaluate(train_data, test_data,
                batch_size=batch_size * config['gpu_num'],
                epochs=num_epochs,
                hooks=[logging_hook])

2020-12-18 16:25:10,582  single_worker_graph_copy_trainer.py (150) (MainThread) : INFO  tasks: ['student_self_trainig']
2020-12-18 16:25:10,584  single_worker_graph_copy_trainer.py (151) (MainThread) : INFO  loss: Tensor("Identity:0", shape=(), dtype=float32, device=/replica:0/task:0/device:CPU:0), loss_list: [<tf.Tensor 'Identity:0' shape=() dtype=float32>], task name: student_self_trainig
2020-12-18 16:25:10,584  single_worker_graph_copy_trainer.py (156) (MainThread) : INFO  do_test: True
2020-12-18 16:25:10,585  single_worker_graph_copy_trainer.py (157) (MainThread) : INFO  building dataset ...


sup_data_size: 9984, unsup_data_size: 399984


2020-12-18 16:25:10,848  single_worker_graph_copy_trainer.py (174) (MainThread) : INFO  start training sess run ...


last: {'unsup_loss': 0.59818876, 'sup_loss': 0.6949824}


task:student_self_trainig, lr:1.0000e-03, loss:0.8161, metric:0.7812:  10%|▉         | 15/156 [00:00<00:00, 141.66it/s]

...

task:student_self_trainig, lr:1.0000e-03, loss:0.4172, metric:1.0000:  10%|▉         | 15/156 [00:00<00:00, 141.79it/s]

'epoch 147 train time(sec): 1.0992083549499512'
{'best_train_metrics': [0.9971],
 'best_validate_metrics': [0.8417],
 'epoch_index': 147,
 'train_loss': [0.3555],
 'train_metrics': [0.9960],
 'validate_loss': [0.7911],
 'validate_metrics': [0.8401]}


task:student_self_trainig, lr:1.0000e-03, loss:0.4872, metric:1.0000:   0%|          | 0/156 [00:00<?, ?it/s]           

last: {'unsup_loss': 0.18010338, 'sup_loss': 0.039812468}
'epoch 148 train time(sec): 1.0823240280151367'
{'best_train_metrics': [0.9971],
 'best_validate_metrics': [0.8417],
 'epoch_index': 148,
 'train_loss': [0.3615],
 'train_metrics': [0.9946],
 'validate_loss': [0.7904],
 'validate_metrics': [0.8398]}


last: {'unsup_loss': 0.28680617, 'sup_loss': 0.038026765}
'epoch 149 train time(sec): 1.0631253719329834'
{'best_train_metrics': [0.9971],
 'best_validate_metrics': [0.8417],
 'epoch_index': 149,
 'train_loss': [0.3693],
 'train_metrics': [0.9949],
 'validate_loss': [0.7796],
 'validate_metrics': [0.8397]}


In my experiment, with 10000 training data of `female_vulgar`, the validate accuracy improved from **0.8259** to **0.8417**.